In [19]:
%pylab inline

import matplotlib.pyplot as plt
import requests
import zipfile
import re
from collections import Counter

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator, GloVe, vocab


from functools import partial,reduce
from tqdm import tqdm, trange
tqdm = partial(tqdm, position=0, leave=True)
trange = partial(trange, position=0, leave=True)

import numpy as np


DEVICE = 'cuda:0'

Populating the interactive namespace from numpy and matplotlib


In [20]:
url = 'http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip'

r = requests.get(url)

ul = url.split('/')
name = ul[len(ul) - 1]

with open(name, 'wb') as file:
  file.write(r.content)

with zipfile.ZipFile(name, "r") as zip_ref:
  zip_ref.extractall("./")

!mv 'cornell movie-dialogs corpus' 'data'
!ls 'data'

 chameleons.pdf			 movie_lines.txt
'cornell movie-dialogs corpus'	 movie_titles_metadata.txt
 movie_characters_metadata.txt	 raw_script_urls.txt
 movie_conversations.txt	 README.txt


In [21]:
FIELD_SPLITTER = '+++$+++'

MAX_SAMPLES = 50000
MAX_LENGTH = 40

UNK_TOKEN = '<unk>'
PAD_TOKEN = '<PAD>'
BOS_TOKEN = '<BOS>'
EOS_TOKEN = '<EOS>'

UNK_TOKEN_IND = 0
PAD_TOKEN_IND = 1
BOS_TOKEN_IND = 2
EOS_TOKEN_IND = 3

BATCH = 2

In [22]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)
    sentence = sentence.strip()
    return sentence

In [23]:
text_transform = lambda x, voc, tokenizer: [voc['<BOS>']] + [voc[token] for token in tokenizer(x)] + [voc['<EOS>']]

In [24]:
# this is terrible as fuck because torchtext is terrible as fuck
def load_conversations(path_to_movie_lines, path_to_movie_conversations):
    id2line = {}
    with open(path_to_movie_lines, errors='ignore') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.replace('\n', '').split(' +++$+++ ')
            id2line[parts[0]] = parts[4]

    inputs, outputs = [], []
    with open(path_to_movie_conversations, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.replace('\n', '').split(' +++$+++ ')
            conversation = [line[1:-1] for line in parts[3][1:-1].split(', ')]
            for i in range(len(conversation) - 1):
                inputs.append(preprocess_sentence(id2line[conversation[i]]))
                outputs.append(preprocess_sentence(id2line[conversation[i + 1]]))
                if len(inputs) >= MAX_SAMPLES:
                    return inputs, outputs
    return inputs, outputs


In [25]:
def get_dataloader(path_to_movie_lines,
                   path_to_movie_conversations):
    questions, answers = load_conversations(path_to_movie_lines, path_to_movie_conversations)

    tokenizer = get_tokenizer('basic_english')

    counter = Counter()
    for sent in questions + answers:
        counter.update(tokenizer(sent))

    voc = vocab(counter)
    voc.insert_token(token=UNK_TOKEN, index=UNK_TOKEN_IND)
    voc.set_default_index(index=UNK_TOKEN_IND)
    voc.insert_token(token=PAD_TOKEN, index=PAD_TOKEN_IND)
    voc.insert_token(token=BOS_TOKEN, index=BOS_TOKEN_IND)
    voc.insert_token(token=EOS_TOKEN, index=EOS_TOKEN_IND)

    q_tokenized = [text_transform(t, voc, tokenizer) for t in questions]
    a_tokenized = [text_transform(t, voc, tokenizer) for t in answers]

    import tensorflow as tf # todo
    q_padded = tf.keras.preprocessing.sequence.pad_sequences(
        q_tokenized, maxlen=MAX_LENGTH, padding='post', value=1.0)

    a_padded = tf.keras.preprocessing.sequence.pad_sequences(
        a_tokenized, maxlen=MAX_LENGTH, padding='post', value=1.0)

    print("Vocab len", len(voc))

    dataloader = DataLoader(
        list(
            zip(
                  q_padded.astype(np.float32),
                  a_padded.astype(np.float32),
                )
            ),
            batch_size=BATCH,
            shuffle=False,
    )

    print(voc)
    torch.save(voc, 'vocab')

    return dataloader, text_transform, voc


In [26]:
lines_path = 'data/movie_lines.txt'
conversations_path = 'data/movie_conversations.txt'    

dataloader, text_transform, voc = get_dataloader(lines_path,
                                                 conversations_path)

Vocab len 23068
Vocab()


In [27]:
test_sample = None

for i,x in enumerate(dataloader):
  if i > 1: break
  print(x[0].shape)
  test_sample = x[0]

torch.Size([2, 40])
torch.Size([2, 40])


In [154]:
# https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fwww.oreilly.com%2Flibrary%2Fview%2Fneural-networks-and%2F9781492037354%2Fassets%2Fmlst_1413.png&f=1&nofb=1
class LSTM(nn.Module):
  def __init__(self, isize, osize):
    super(LSTM, self).__init__()

    self.isize = isize
    self.osize = osize

    self.f = nn.Linear(isize, osize)
    self.g = nn.Linear(isize, osize)
    self.i = nn.Linear(isize, osize)
    self.o = nn.Linear(isize, osize)

    self.tanh = nn.Tanh()
    self.sigmoid = nn.Sigmoid()

  def forget_gate(self, h, x, c):
    fho = self.f(h)
    fxo = self.f(x)

    fo = fho + fxo
    fos = self.sigmoid(fo)
    out = torch.mul(c, fos)

    return out

  def input_gate(self, h, x):
    gho = self.g(h)
    gxo = self.g(x)
    go = gho + gxo
    gto = self.tanh(go)

    iho = self.i(h)
    ixo = self.i(x)
    io = iho + ixo
    ito = self.sigmoid(io)

    print(gto.shape, ito.shape)
    out = torch.mul(gto, ito)

    return out

  def output_gate(self, h, x, cn):
    oho = self.o(h)
    oxo = self.o(x)
    oo = oho + oxo

    out = torch.mul(oo, cn)

    return out

  def forward(self, x, h0=None, c0=None):
    if not h0:
      h0 = torch.zeros(x.shape[0], self.isize)
    if not c0:
      c0 = torch.zeros(x.shape[0], self.osize)

    fg_out = self.forget_gate(h0, x, c0)
    ig_out = self.input_gate(h0, x)

    c_new = fg_out + ig_out

    cnt = self.tanh(c_new) 

    h_new = self.output_gate(h0, x, c_new)

    out = h_new

    return out, (h_new, c_new)


In [155]:
my_lstm = LSTM(40, 80)

out, (h, c) = my_lstm(test_sample)

print(out.shape)
print(out)
print('\n----------------')
print(h.shape)
print(h)
print('\n----------------')
print(c.shape)
print(c)

torch.Size([2, 80]) torch.Size([2, 80])
torch.Size([2, 80])
tensor([[-1.4674e-03, -5.0430e+00,  9.3533e+00,  1.9533e+00,  1.1137e+00,
         -2.3917e-06,  7.2047e+00, -3.4163e-01,  3.8996e+00, -1.6308e-03,
          9.2821e+00, -1.0249e+01, -2.0752e+01, -8.0536e-03, -1.6357e+01,
         -8.8634e+00, -1.1861e+01,  4.9321e+00, -6.6905e+00, -9.5990e+00,
          5.8808e+00,  1.0157e-06, -5.1388e+00,  2.2135e+00,  3.7224e-02,
         -5.2040e-05, -3.8808e+00,  7.4679e-05,  3.3302e+00, -2.6798e-07,
         -4.3820e-02,  9.9563e+00, -1.2414e+00,  2.7979e-01,  2.7267e+00,
          3.3662e+00,  4.5779e+00,  1.6907e+00,  2.1736e-03, -1.5878e-01,
          1.8046e-06,  9.1593e-02, -1.3679e+00,  4.4969e+00,  1.5936e-08,
         -8.8859e-08,  1.1087e+01,  1.4038e+01, -2.1818e+01,  4.9290e+00,
          9.8502e+00, -2.4420e-08, -1.7005e-10,  9.6493e+00,  8.2473e-01,
         -1.4403e-01, -1.0829e-06, -3.9287e-04, -2.1566e+01, -4.4556e+00,
         -1.6597e+01, -3.5435e+00,  5.0527e-05, -2.2

In [37]:
print(test_sample.shape)
print(test_sample.dtype 

torch.Size([2, 40])
torch.float32


In [156]:
t_lstm = nn.LSTM(40, 80, batch_first=True)

res, (h_s, c_s) = t_lstm(test_sample)

print(res.shape)
print(res)
print('\n---------')
print(h_s.shape)
print(h_s)
print('\n---------')
print(c_s.shape)
print(c_s)
print('\n---------')

torch.Size([2, 80])
tensor([[-3.5065e-03, -1.9093e-05,  9.2510e-06,  9.8781e-05, -5.7811e-01,
         -3.2824e-08,  3.6725e-02, -7.2311e-01,  1.0724e-02, -3.1983e-01,
         -2.3204e-01,  2.7326e-01, -1.1981e-01,  6.8479e-01,  8.7582e-03,
          1.4554e-04,  2.0267e-04, -6.2998e-06,  5.9079e-04, -3.0727e-01,
          3.1095e-03, -1.9977e-05,  3.3266e-05,  7.1113e-04, -4.0071e-03,
         -3.6446e-12,  5.4377e-01, -7.6159e-01, -7.5062e-04, -6.0906e-04,
          7.8045e-08,  1.3249e-01,  7.6098e-01,  3.9227e-02,  6.1729e-01,
          2.1250e-05,  3.3744e-01, -5.7826e-06,  3.1647e-04, -4.3004e-13,
         -3.7966e-05, -3.4999e-01, -1.0875e-01, -6.4876e-04, -6.6209e-01,
          1.4063e-01, -3.2276e-05, -2.8892e-09, -7.2229e-08, -1.4007e-01,
          1.4035e-02, -6.8325e-03, -8.6691e-03, -4.0947e-05, -7.5320e-01,
          1.3997e-03, -7.4605e-01, -6.9942e-01,  7.5422e-01,  7.2532e-01,
          6.0891e-01, -2.8445e-01, -6.8476e-01,  9.3698e-05,  1.1946e-04,
         -5.8461e-

In [ ]:
!cat data/movie_conversations.txt | tail -n 10

In [ ]:
!ls data

In [ ]:
!cat data/README.txt

In [ ]:
# # trash




# def preprocess(x):
#   x_no_new = x.replace('\n', '')
#   text = x_no_new.split(FIELD_SPLITTER).pop()
#   embedding = g_vectors.get_vecs_by_tokens(tokenizer(text), lower_case_backup=True)
#   return embedding

# tokenizer = get_tokenizer('basic_english')
# g_vectors = GloVe(name='840B')
# g_vocab = vocab(g_vectors.stoi)


# train_iter = tt.data.BucketIterator(
#   dataset=train_obj,
#   batch_size = 2,
#   sort_key=lambda x: len(x.review),
#   shuffle=True,
#   device=DEVICE
# )

# trainloader = torch.utils.data.DataLoader(
# 	,
# 	batch_size=BATCH,
# 	num_workers=12,
# 	shuffle=True
# )

In [ ]:

# embeddings = global_vectors.get_vecs_by_tokens(tokenizer("Hello, How are you?"),
#                                                lower_case_backup=True)
# embeddings
# 
# 
# 
# def batch(iterable, size):
#     from itertools import chain, islice
#     iterator = iter(iterable)
#     for first in iterator:
#         yield list(chain([first], islice(iterator, size - 1)))